In [1]:
import sys, os
import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt
# import gensim
import pytorch_pretrained_bert as ppb
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
# import logging
# logging.basicConfig(level=logging.INFO)
# from glove import Glove
# from glove import Corpus
import util
import csv
from tqdm import tqdm

import word_util as wtil
import graph_util as gtil

# import torch
import spacy
nlp = spacy.load('en')

In [2]:
questions = wtil.load_questions('../aristo-mini/questions/AI2-Elementary-NDMC-Feb2016-Train.jsonl')
len(questions)

432

In [3]:
device = 'cpu'

# init_checkpoint_pt = '/mnt/c/Users/anwan/OneDrive - UW Office 365/Khan/Capstone/pytorch-pretrained-BERT/uncased_L-12_H-768_A-12.zip'

In [4]:
model = BertModel.from_pretrained('bert-base-uncased')
model.eval()
model.to(device)

modelp = BertForMaskedLM.from_pretrained('bert-base-uncased')
modelp.eval()
modelp.to(device)
pass

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def format_question(q, ans):
    opts = []
    
    for a in ans:
        x = '[CLS] ' + q.replace('?', ' ? ').replace('.', ' . ') + ' [SEP] ' + a + ' [SEP]'
        
        toks = tokenizer.tokenize(x)
#         print(toks)
        
        idx = toks.index('[SEP]')
        seg = [0]*(idx + 1) + [1]*(len(toks)-idx-1)
        
        tokens_tensor = torch.tensor([tokenizer.convert_tokens_to_ids(toks)])
        segments_tensors = torch.tensor([seg])

        tokens_tensor = tokens_tensor.to(device)
        segments_tensors = segments_tensors.to(device)
        
        with torch.no_grad():
            encoded_layers, _ = model(tokens_tensor, segments_tensors)
        # We have a hidden states for each of the 12 layers in model bert-base-uncased
        embs = encoded_layers[-1][0]
        
        opts.append(embs[segments_tensors.byte().squeeze()])
    
    x = '[CLS] ' + q.replace('?', ' ? ').replace('.', ' . ') + ' [SEP]'
    toks = tokenizer.tokenize(x)
    
    seg = [0]*(len(toks))
    
    tokens_tensor = torch.tensor([tokenizer.convert_tokens_to_ids(toks)])
    segments_tensors = torch.tensor([seg])
    
    with torch.no_grad():
        encoded_layers, _ = model(tokens_tensor, segments_tensors)
    # We have a hidden states for each of the 12 layers in model bert-base-uncased
    embs = encoded_layers[-1][0]
    
    return embs, opts



In [8]:
i = 1
q, opts = format_question(questions[i]['question']['stem'], [c['text'] for c in questions[i]['question']['choices']])
q.shape, [o.shape for o in opts]

(torch.Size([46, 768]),
 [torch.Size([2, 768]),
  torch.Size([2, 768]),
  torch.Size([2, 768]),
  torch.Size([2, 768])])

In [15]:
path = 'aristo.txt'

In [22]:
G = util.adict()
iG = util.adict()

In [23]:
with open(path, 'r') as tsvfile:
    reader = csv.reader(tsvfile, delimiter='\t')
    for i, row in tqdm(enumerate(reader)):
        sub, rel, obj = row
        if sub not in G:
            G[sub] = util.adict()
        if rel not in G[sub]:
            G[sub][rel] = set()
        G[sub][rel].add(obj)
        if obj not in iG:
            iG[obj] = util.adict()
        if rel not in iG[obj]:
            iG[obj][rel] = set()
        iG[obj][rel].add(sub)

282594it [00:02, 117608.85it/s]


In [30]:
total = 0
for sub, rels in G.items():
    for rel, objs in G[sub].items():
        for obj in objs:
            total += 1
total

282594

In [51]:
rows = []
with open(path, 'r') as tsvfile:
    reader = csv.reader(tsvfile, delimiter='\t')
    for i, row in enumerate(reader):
            
        rows.append(row)
len(rows)

282594

In [61]:
step = 1000
idx = 0
print(len(rows)//step)

282


In [60]:
tpls = {}
for idx in range(len(rows)//step + 1):
    for i in range(idx*step,(idx+1)*step):
        row = rows[i]
        x = ' '.join(row)

        toks = tokenizer.tokenize(x)

        toks = ['[CLS]'] + toks + ['[SEP]']

        seg = [0]*len(toks)

        tokens_tensor = torch.tensor([tokenizer.convert_tokens_to_ids(toks)])
        segments_tensors = torch.tensor([seg])

        tokens_tensor = tokens_tensor.to(device)
        segments_tensors = segments_tensors.to(device)

        with torch.no_grad():
            encoded_layers, _ = model(tokens_tensor, segments_tensors)
        # We have a hidden states for each of the 12 layers in model bert-base-uncased
        embs = encoded_layers[-1][0]

        tpls[tuple(row)] = embs

        if i % (len(rows)//1000) == 0:
            print('{}/{} : {}'.format(i+1, len(rows), len(tpls)))

        if i == 100:
            break

    torch.save(tpls, 'table/entries_{}.pth.tar'.format(str(idx).zfill(5)))
    print(idx+1, len(rows)//step)

1/282594 : 1
1 5651
2 5651
3 5651
4 5651
5 5651
283/282594 : 234
6 5651


KeyboardInterrupt: 

In [56]:
torch.save?

In [53]:
len(tpls)

101